In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [5]:
analize = Analizer(0.7)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="weights.h5", remove_last=False)



/tmp/ipykernel_5995/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_7_11_7,0.999519,0.840173,0.871310,0.995883,0.002020,1.068764,0.460317,0.009648,0.042051,0.044943,1.000282,0.046856,142.409470,221.636398,"Hidden Size=[8, 4], regularizer=0.02, learning..."
1,model_7_11_3,0.999559,0.840162,0.872030,0.996496,0.001852,1.068835,0.457740,0.008212,0.045520,0.043038,1.000258,0.044870,142.582717,221.809646,"Hidden Size=[8, 4], regularizer=0.02, learning..."
2,model_7_11_0,0.999582,0.840138,0.872791,0.997101,0.001754,1.069001,0.455019,0.006794,0.049186,0.041878,1.000245,0.043661,142.692004,221.918933,"Hidden Size=[8, 4], regularizer=0.02, learning..."
3,model_7_7_7,0.994748,0.840073,0.884625,0.971975,0.022043,1.069436,0.412689,0.092651,0.212314,0.148470,1.003075,0.154790,137.629501,216.856430,"Hidden Size=[8, 4], regularizer=0.02, learning..."
4,model_7_12_2,0.999654,0.839948,0.870549,0.997290,0.001454,1.070265,0.463039,0.001852,0.040897,0.038128,1.000203,0.039752,143.067177,222.294105,"Hidden Size=[8, 4], regularizer=0.02, learning..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,model_6_5_0,0.698357,0.502642,0.609859,0.614413,1.265932,3.325835,1.395513,0.624225,1.887582,1.125137,1.176572,1.173036,129.528382,208.755311,"Hidden Size=[8, 4], regularizer=0.2, learning_..."
709,model_16_0_0,0.707991,0.501716,0.668112,0.505098,1.225501,3.332028,1.187144,1.494933,1.462444,1.107023,1.019413,1.154152,769.593301,1238.860493,"Hidden Size=[24, 12], regularizer=0.02, learni..."
710,model_3_9_2,0.909129,0.501513,0.532844,0.733353,0.381367,3.333385,1.670989,0.993974,1.330609,0.617549,3.180911,0.643840,51.927984,82.399880,"Hidden Size=[4, 2], regularizer=0.02, learning..."
711,model_10_5_0,0.698942,0.500844,0.523817,0.749696,1.263477,3.337862,1.703280,0.974875,2.206431,1.124045,1.074489,1.171898,241.532265,389.016240,"Hidden Size=[12, 6], regularizer=0.2, learning..."
